# Домашняя работа к лекции «Проблемы качества данных»

In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from IPython.display import Image

In [2]:
HOUSING_PATH = "housing.csv"

In [3]:
housing = pd.read_csv(HOUSING_PATH)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


**Масштабирование:**

а. Нормализовать признак - median_income<br>
б. Привести признак - population к масштабу от 0 до 100

In [4]:
# а. Нормализовать признак - median_income
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
x_scaled = min_max_scaler.fit_transform(housing[['median_income']].values)
housing_temp = pd.DataFrame(x_scaled, columns=['median_income'], index = housing.index)
housing[['median_income']] = housing_temp

In [5]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,0.539668,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,0.538027,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,0.466028,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,0.354699,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,0.230776,342200.0,NEAR BAY


In [6]:
# б. Привести признак - population к масштабу от 0 до 100

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
x_scaled = min_max_scaler.fit_transform(housing[['population']].values)
housing_temp = pd.DataFrame(x_scaled, columns=['population'], index = housing.index)
housing[['population']] = housing_temp

In [7]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,0.894083,126.0,0.539668,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,6.721040,1138.0,0.538027,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,1.381765,177.0,0.466028,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,1.555537,219.0,0.354699,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,1.575156,259.0,0.230776,342200.0,NEAR BAY


In [8]:
housing.population.min()

0.0

In [9]:
housing.population.max()

100.0

**Заполнение пропусков:**

а. Заполнить пропуски в столбце total_bedrooms средними значениями по категориям столбца ocean_proximity.<br>
Например, если пропущено значение в строке с категорией INLAND, нужно заполнить его средним значением по этой категории

In [10]:
# посмотрим, сколько всего пропусков в столбце
housing['total_bedrooms'].isnull().sum()

207

In [11]:
# выведем первые 5 строк с пропусками в столбце total_bedrooms
housing[housing['total_bedrooms'].isnull()].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,NaN,1.589170,218.0,0.267245,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,NaN,2.043219,259.0,0.077220,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,NaN,10.476751,1273.0,0.143191,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,NaN,1.067855,146.0,0.306823,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,NaN,1.076263,161.0,0.234921,178400.0,NEAR BAY


In [12]:
# заменим пропуски в столбце на среднее по категории столбца ocean_proximity
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(housing.groupby('ocean_proximity')['total_bedrooms']\
                                                             .transform('mean'))

In [13]:
# проверим, что получилось
# средние значения по категориям ocean_proximity для столбца total_bedrooms
housing[['total_bedrooms', 'ocean_proximity']].groupby('ocean_proximity').mean()

,total_bedrooms
ocean_proximity,
<1H OCEAN,546.539185
INLAND,533.881619
ISLAND,420.400000
NEAR BAY,514.182819
NEAR OCEAN,538.615677


In [14]:
# проверим для строки 209 (ранее был пропуск, категория NEAR BAY, следовательно, значение должно быть 514.182819)
housing.iloc[[290]]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,514.182819,1.58917,218.0,0.267245,161900.0,NEAR BAY


In [15]:
# пустых значений теперь 0
housing['total_bedrooms'].isnull().sum()

0

**Категорийные атрибуты:**

а. Посчитать среднее кол-во комнат в строке (total_rooms/households)<br>
б. Разбить полученный столбец на группы на ваше усмотрение (Например, 1 комната, 2 комнаты, 3 и более комнат)

In [16]:
# создаем новый столбец
# round, т.к. количество комнат должно быть дискретным
housing['rooms_per_households'] = round(housing["total_rooms"]/housing["households"])
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_households
0,-122.23,37.88,41.0,880.0,129.0,0.894083,126.0,0.539668,452600.0,NEAR BAY,7.0
1,-122.22,37.86,21.0,7099.0,1106.0,6.721040,1138.0,0.538027,358500.0,NEAR BAY,6.0
2,-122.24,37.85,52.0,1467.0,190.0,1.381765,177.0,0.466028,352100.0,NEAR BAY,8.0
3,-122.25,37.85,52.0,1274.0,235.0,1.555537,219.0,0.354699,341300.0,NEAR BAY,6.0
4,-122.25,37.85,52.0,1627.0,280.0,1.575156,259.0,0.230776,342200.0,NEAR BAY,6.0


In [17]:
# посмотрим, какие значения есть в новом столбце
housing['rooms_per_households'].unique()

array([  7.,   6.,   8.,   5.,   4.,   3.,   2.,   9.,  11.,  30.,  10.,
        32.,  17.,  29.,  13.,  28.,  34.,  48.,  22.,  24.,  39.,  21.,
        12.,  16.,  56.,  62., 142.,  37.,  35., 133.,  26.,  51.,  20.,
        31.,   1.,  18.,  15.,  14.,  53.,  19.,  25.,  36.,  27.,  41.,
        40.,  23.,  60.])

In [18]:
# предлагаю разбить на 8 групп: 0-10, 10-20, 20-30, 30-40, 40-50, 50-60, 60-70, >70 (т.к. между 62 и 133 нет значений)
def replaceRooms(rooms_count):
    return {
        0 <= rooms_count <10 :  '0-10',
        10 <= rooms_count <20 : '10-20',
        20 <= rooms_count <30 : '20-30',
        30 <= rooms_count <40 : '30-40',
        40 <= rooms_count <50 : '40-50',
        50 <= rooms_count <60 : '50-60',
        60 <= rooms_count <70 : '60-70',
        rooms_count >=70 :      '>70'
    }[True]

In [19]:
housing['rooms_per_households'] = housing['rooms_per_households'].map(replaceRooms)

In [20]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_households
0,-122.23,37.88,41.0,880.0,129.0,0.894083,126.0,0.539668,452600.0,NEAR BAY,0-10
1,-122.22,37.86,21.0,7099.0,1106.0,6.721040,1138.0,0.538027,358500.0,NEAR BAY,0-10
2,-122.24,37.85,52.0,1467.0,190.0,1.381765,177.0,0.466028,352100.0,NEAR BAY,0-10
3,-122.25,37.85,52.0,1274.0,235.0,1.555537,219.0,0.354699,341300.0,NEAR BAY,0-10
4,-122.25,37.85,52.0,1627.0,280.0,1.575156,259.0,0.230776,342200.0,NEAR BAY,0-10


In [33]:
housing.iloc[1020:1030]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_households
1020,-121.78,37.66,25.0,1947.0,418.0,2.514084,354.0,0.231197,193000.0,INLAND,0-10
1021,-121.79,37.66,22.0,14701.0,2210.0,18.750526,2232.0,0.377933,245000.0,INLAND,0-10
1022,-119.78,38.69,17.0,1364.0,282.0,0.938928,152.0,0.134488,117600.0,INLAND,0-10
1023,-119.93,38.72,15.0,2061.0,465.0,1.597578,196.0,0.120122,97900.0,INLAND,10-20
1024,-120.00,38.52,16.0,3045.0,543.0,0.557751,102.0,0.182763,140600.0,INLAND,30-40
1025,-120.56,38.48,14.0,3545.0,702.0,2.643011,411.0,0.204204,120900.0,INLAND,0-10
1026,-120.59,38.45,13.0,2944.0,558.0,3.049413,466.0,0.172418,119000.0,INLAND,0-10
1027,-120.55,38.46,16.0,1443.0,249.0,1.210796,181.0,0.186425,129200.0,INLAND,0-10
1028,-120.55,38.45,17.0,2277.0,474.0,2.141316,356.0,0.139370,99100.0,INLAND,0-10
1029,-120.55,38.43,18.0,1564.0,357.0,1.723703,277.0,0.127929,108900.0,INLAND,0-10


In [21]:
# посмотрим, какие значения есть в новом столбце
housing['rooms_per_households'].unique()

array(['0-10', '10-20', '30-40', '20-30', '40-50', '50-60', '60-70',
       '>70'], dtype=object)